<a href="https://colab.research.google.com/github/coneill000/French-Flashcards-with-Anki/blob/file-naming/French_Speech_Recognition_and_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# French Speech Recognition and Translation
Welcome to my speech recognition and translation program. This was designed to speed up the process of making French Anki flashcards by using the audio captured from the textbook. The program will first recognize the French word being spoken and rename the audio file accordingly. It will then use that word recognition to translate the meaning into English. All of this information will be stored in a Google Sheets, where you can manually check any errors, missing vocab words, etc. Yeah it's kind of overly complicated but who cares. 

## Imports
Run this first so you have the necessary imports. The imports are split into two cells, one for pip installs and one for library imports

In [ ]:
!pip install pydub
!pip install SpeechRecognition
!pip3 install googletrans

In [3]:
import os
import re
import gspread
import speech_recognition as sr
from pydub import AudioSegment
from googletrans import Translator, constants
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

## Google Authorization
Run this to give the program access to your Google Sheets. Please make sure that you mount your Google Drive using Google Colab. 

In [4]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
drive.mount('/content/drive')

## User-Set Variables


In [7]:
directory = '/content/drive/My Drive/Ch3Vocab' #change this to directory where audio files are stored
sheetname = 'En avant Ch3Vocab' #change this to desired name of Google Sheets document
tag = 'EnAvantCh3' #change this to the desired tag(s) you want on the cards

## Main Program (aka Under the Hood)

In [8]:
successpath = os.path.join(directory, 'success') 
errorpath = os.path.join(directory, 'error')
os.mkdir(successpath)
os.mkdir(errorpath)

In [9]:
r = sr.Recognizer()
translator = Translator()
frenchlist = []

In [12]:
for filename in os.listdir(directory):
    is_match = bool(re.match("S.mp3|S-\d.mp3|S-\d\d.mp3|S-\d\d\d.mp3", filename))
    if is_match:
        src = f'{directory}/{filename}'
        dst = f'{directory}/{filename[:-4]}.wav'
        sound = AudioSegment.from_mp3(src)
        sound.export(dst, format="wav")
        temp = sr.AudioFile(dst)
        with temp as source:
            audio = r.record(source)
        try:
            french = r.recognize_google(audio, language='fr-FR')
            if "'" in french:
                french = french.replace("'", "_")
            os.rename(src, f'{successpath}/{french}.mp3')
            frenchlist.append(french)
        except:
            os.rename(src, f'{errorpath}/{filename}')

In [13]:
#sets up sheet/opens pre-existing sheet for info storage
sh = gc.create(sheetname)
worksheet = gc.open(sheetname).sheet1

In [14]:
#sets up basic worksheet information
worksheet.update_cell(1, 1, "French")
worksheet.update_cell(1, 2, "English")
worksheet.update_cell(1, 3, "Audio")
worksheet.update_cell(1, 4, "Tags")

num = len(frenchlist) + 1
french_cells = worksheet.range(f'A2:A{num}')
meaning_cells = worksheet.range(f'B2:B{num}')
audio_cells = worksheet.range(f'C2:C{num}')
tag_cells = worksheet.range(f'D2:D{num}')

In [ ]:
#fills in Google Sheet with information
for i, cell in enumerate(french_cells):
    cell.value = frenchlist[i]

for i, cell in enumerate(meaning_cells):
    translation = translator.translate(frenchlist[i], src="fr")
    cell.value = translation.text

for i, cell in enumerate(audio_cells):
    cell.value = f"[sound:{frenchlist[i]}.mp3]"

for i, cell in enumerate(tag_cells):
    cell.value = tag

worksheet.update_cells(french_cells)
worksheet.update_cells(meaning_cells)
worksheet.update_cells(audio_cells)
worksheet.update_cells(tag_cells)